# Data transformation query

Hereby a "checklist" for what needs to be done before it can be put into a model:<br><br>
- remove outliers <br>
- create lags of relevant fields (product-specific demand is specifically interesting)<br>
- make a variable that counts the number of weeks from the start of the set (as a numerical inflation variable)<br>
- remove zero demand observations (after creating lags -> ptherwise lags will not be accurate. zero demand in lag will be handled differently *)<br>
- log-transform all the numerical variables (after removing zero demand observations --> otherwise they become -infinity)<br>
- replace the lagged variables that were zero and are now log-transformed to value -infinity (*) with a mean (or max in case the zero demand is due to promo) value of demand<br>
- create dummy (0/1) variables out of the categorical features using a OneHotEncoder or pd.get_dummies (and leave 1 default dummy per categorical variable out: if you include all dummies you cannot work with it due to rank deficiency)<br>
- include a constant<br><bar>
So then all the relevant variables are ready and transformed; and it's important that all input variables are numerical (either a log-transformed value or a dummy variable) and there are no -infinity values left. <br>
<br>
Note: NaN values are okay for XGBoost models - but not for others such as Random Forest<br>
potentially: you can include nonlinear terms such as squared values of variables (f.e. the aforementioned inflation variable, unavailability percentage, anything that makes sense in your opinion)

In [1]:
import numpy as np
import pandas as pd
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from sklearn.model_selection import train_test_split
# import sklearn
import statsmodels.api 
import seaborn as sns
from dateutil import parser
from sklearn.preprocessing import PolynomialFeatures
from datetime import datetime


In [2]:
pd.options.display.max_rows = 4000
pd.options.display.max_columns = 4000

In [3]:
# #in case after transformations PPL is still wrong:
# df_log=pd.read_csv("dataframe_transformed_4_log.csv")
# new_col = pd.Series(['PPL1', 'PPL2', 'PPL3'] * (df_log.shape[0] // 3 + 1))[:df_log.shape[0]]
# df_log['PPL']=new_col
# df_log = pd.get_dummies(df_log, columns=['PPL'])
# df_log['postShift'] = df_log['WEEK_START_DATE'] > "2021-06-20"
# df_log['postPPL1'] = df_log['postShift']*df_log['PPL_PPL1']
# df_log['postPPL2'] = df_log['postShift']*df_log['PPL_PPL2']
# df_log['postPPL3'] = df_log['postShift']*df_log['PPL_PPL3']
# df_log['PPL1'] = df_log['PPL_PPL1']-df_log['postPPL1']
# df_log['PPL2'] = df_log['PPL_PPL2']-df_log['postPPL2']
# df_log['PPL3'] = df_log['PPL_PPL3']-df_log['postPPL3']
# df_log.to_csv('dataframe_transformed_5_log.csv') # opnieuw opslaan - het klopte niet

In [4]:
df=pd.read_csv("DF_WEEK_DRINKS_2023.csv")
# df=df.drop('#',axis=1)
df=df.set_index(['KEY_ARTICLE', 'KEY_WEEK','PPL'])

In [6]:
# remove nans with zero
df[['product order amt','AVG unavailability_perc']]=df[['product order amt','AVG unavailability_perc']].fillna(0)

In [1]:
#function to make lagged variables (can also be done in SQL query)
def buildLaggedFeatures(s,lag=2,dropna=False):

# Builds a new DataFrame to facilitate regressing over all possible lagged features


    if type(s) is pd.Series:
        the_range=range(lag+1)
        res=pd.concat([s.groupby(['KEY_ARTICLE','PPL']).shift(i) for i in the_range],axis=1)
        res.columns=['lag_%d' %i for i in the_range]
    # elif type(s) is pd.DataFrame:
    #     new_dict={}
    #     for col_name in s:
    #         new_dict[col_name]=s[col_name]
    #         # create lagged Series
    #         for l in range(1,lag+1):
    #             new_dict['%s_lag%d' %(col_name,l)]=s[col_name].groupby(['KEY_ARTICLE','PPL']).shift(l) 
    #     res=pd.DataFrame(new_dict,index=s.index)
    else:
        print('Only works for (DataFrame or) Series')
        return None
    if dropna:
        return res.dropna()
    else:
        return res 

In [9]:
# make lags
df_orderamt=buildLaggedFeatures(df['product order amt'],lag=2,dropna=False)
df_orderamt.set_axis(['product order amt', 'product order amt_t-1', 'product order amt_t-2'], axis=1, inplace=True)
df_price=buildLaggedFeatures(df['Avg sell price'],lag=2,dropna=False)
df_price.set_axis(['Avg sell price', 'Avg sell price_t-1', 'Avg sell price_t-2'], axis=1, inplace=True)

df[['product order amt_t-1', 'product order amt_t-2']] = df_orderamt[['product order amt_t-1', 'product order amt_t-2']]
df[[ 'Avg sell price_t-1', 'Avg sell price_t-2']]=df_price[['Avg sell price_t-1', 'Avg sell price_t-2']]

In [13]:
#choose columns to include in dataframe
vars= df[['product order amt', 'Avg sell price',
       'Avg purchase price', 
        'AVG unavailability_perc',
       'AVG_HIGH_TEMP', 'total order amt', 
       'NR_ARTICLES_IN_CAT','NR_ARTICLES_IN_CAT_2',
       'product order amt_t-1', 'product order amt_t-2', 'Avg sell price_t-1',
       'Avg sell price_t-2', 'YEAR_CALENDAR_WEEK', 'WEEK_START_DATE','ARTICLE_NAME','PROMO_DUMMY', 'ARTICLE_TIER', 'ART_BRAND_TIER', 'Packaging', 'ARTICLE_CAT_2', 'ARTICLE_CAT_3',
       'ARTICLE_CAT_4', 'ARTICLE_ID','ART_CONTENT_VOLUME',
       'ART_IS_MULTIPACK',]] 
# exclude cat3 (4)  (out of scope for now)
# because Dummies cat3 (4) hade to be made within each cat2 (3): because per category level, for each ``group'' of subcategories that form a category: you need to remove one of 
# the subcategories for regression to be executed (rank deficiency issues occur in case all categories are included)

In [14]:
# after creating lags: remove first 2 weeks
# make dataset start from week 53 on
vars = vars[vars['product order amt_t-2'].notna()] 

In [17]:
#make dummies 

vars.reset_index('PPL', inplace=True) # you want PPL in dummified form
dum_cols=['PPL','ARTICLE_TIER', 'ART_BRAND_TIER', 'Packaging', 'ARTICLE_CAT_2', 'ARTICLE_CAT_3', 'ARTICLE_CAT_4']
print('Default dummy values are: ',(set(pd.get_dummies(vars,columns=dum_cols))).difference(set(pd.get_dummies(vars,columns=dum_cols,drop_first=True))))
vars=pd.get_dummies(vars.join(vars0['ARTICLE_CAT_2'],rsuffix='_complete'),columns=dum_cols,drop_first=True)
# vars=sm.tools.tools.add_constant(vars)
df=vars.copy()

Default dummy values are:  {'PPL_postPPL1', 'ARTICLE_TIER_Best', 'ART_BRAND_TIER_A-brand', 'ARTICLE_CAT_2_Cola', 'ARTICLE_CAT_4_Appelsap', 'Packaging_Bottle', 'ARTICLE_CAT_3_Appelsap'}


In [21]:
df.reset_index(['KEY_ARTICLE', 'KEY_WEEK'],inplace=True)

In [22]:
# make week count number variable (inflatie / time measure)
df.WEEK_START_DATE = df.WEEK_START_DATE.apply(lambda x: parser.parse(x))
date_string = "2020-12-28"
startDate = datetime.strptime(date_string, '%Y-%m-%d')
df['DAYS_SINCE_START'] = df.WEEK_START_DATE.apply(lambda x: x - startDate)
df['WEEKS_SINCE_START'] = df.DAYS_SINCE_START.apply(lambda x: x.days/7)

df['WEEK_NR'] = df.WEEKS_SINCE_START.apply(lambda x: x + 1)
df['AVG unavailability_perc']=df['AVG unavailability_perc'].apply(lambda x: x*100)
df[['product order amt','AVG unavailability_perc']]=df[['product order amt','AVG unavailability_perc']].fillna(0)
df[['KEY_ARTICLE', 'KEY_WEEK', 'YEAR_CALENDAR_WEEK','ARTICLE_NAME','ARTICLE_ID']]=df[['KEY_ARTICLE', 'KEY_WEEK', 'YEAR_CALENDAR_WEEK','ARTICLE_NAME','ARTICLE_ID']].astype(str)

In [48]:
# apply log-transformation on subset of columns: can only apply np.log() on numerical features 

logdf=df[['KEY_ARTICLE','KEY_WEEK','product order amt','Avg sell price', 'Avg purchase price', 'AVG_HIGH_TEMP', 'total order amt','NR_ARTICLES_IN_CAT',
'NR_ARTICLES_IN_CAT_2', 'product order amt_t-1',
'product order amt_t-2', 'Avg sell price_t-1','Avg sell price_t-2', 'WEEK_NR','AVG unavailability_perc',
]]
logdf.set_index(['KEY_ARTICLE','KEY_WEEK'],inplace=True) # used to join with df
logdf.rename(columns=lambda x: 'l_'+x, inplace=True)
logdf=np.log(logdf)
df_log=pd.concat([df,logdf],axis=1,)

C:\Users\Florence Piet\AppData\Local\Temp\ipykernel_19088\4175568398.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logdf.rename(columns=lambda x: 'l_'+x, inplace=True)
c:\Apps\Anaconda\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [ ]:
#include a constant
df_log=statsmodels.tools.tools.add_constant(df_log)

In [54]:
# Save transformed set
df_log.to_csv('dataframe_transformed_2023.csv')

Deprecated:

In [ ]:
#making polynomial features to include nonlinearity in linear models like Lasso

# Set up set with polynomials
data=df_log2.copy()
data=data.reset_index()
features = data.copy()[['l_Avg sell price','l_Avg purchase price','l_AVG_HIGH_TEMP', 'l_total order amt','l_NR_ARTICLES_IN_CAT', 'l_NR_ARTICLES_IN_CAT_2',
# 'l_product order amt_t-1','l_product order amt_t-2', 'l_Avg sell price_t-1','l_Avg sell price_t-2', 
'l_WEEK_NR', 'AVG unavailability_perc']]

poly_dict = {'l_Avg sell price':2,
'l_Avg purchase price':2,
'l_AVG_HIGH_TEMP':2,
'l_total order amt':2,
'l_NR_ARTICLES_IN_CAT':2,
'l_NR_ARTICLES_IN_CAT_2':2,
# 'l_product order amt_t-1':2,
# 'l_product order amt_t-2':2,
# 'l_Avg sell price_t-1':2,
# 'l_Avg sell price_t-2':2,
'l_WEEK_NR':2,
'AVG unavailability_perc':2}

for key, degree in poly_dict.items():
    poly = PolynomialFeatures(degree, include_bias=False)
    data_transf = poly.fit_transform(data[[key]])
    x_cols = poly.get_feature_names_out([key])
    data_transf = pd.DataFrame(data_transf, columns=x_cols)

    features = pd.concat((features, data_transf),
                          axis=1, sort=False,copy=False)

df_log3 = pd.concat((data.copy()[['KEY_ARTICLE','KEY_WEEK','l_product order amt','const', 
      'l_product order amt_t-1','l_Avg sell price_t-1','l_product order amt_t-2','l_Avg sell price_t-2',
       'PPL2', 'PPL3',
       'YEAR_CALENDAR_WEEK', 'WEEK_START_DATE', 'ARTICLE_NAME', 'ARTICLE_ID',
       'ARTICLE_TIER_Better', 'ARTICLE_TIER_Good',
       'ART_BRAND_TIER_Price entry', 'ART_BRAND_TIER_Private label',
       'Packaging_Box', 'Packaging_Can', 'Packaging_Pack','ARTICLE_CAT_2_complete',
       'ARTICLE_CAT_2_Drinkpakjes', 'ARTICLE_CAT_2_Fruitdrank',
       'ARTICLE_CAT_2_IJsthee', 'ARTICLE_CAT_2_Limonade & siropen',
       'ARTICLE_CAT_2_Sappen & smoothies',
       'ARTICLE_CAT_2_Sinas, Lemon & Cassis', 'ARTICLE_CAT_2_Speciaal fris',
       'ARTICLE_CAT_2_Sport- & energydrink', 'ARTICLE_CAT_2_Water',
       'PROMO_DUMMY', 'PPL1', 'postPPL1', 'postPPL2', 'postPPL3']], features.iloc[:,8:].copy()[['l_Avg sell price',
       'l_Avg sell price^2', 'l_Avg purchase price', 'l_Avg purchase price^2',
       'l_AVG_HIGH_TEMP', 'l_AVG_HIGH_TEMP^2', 'l_total order amt',
       'l_total order amt^2',
    #    'l_total order amt_t-1',
    #    'l_total order amt_t-1^2', 'l_total order amt_t-2',
    #    'l_total order amt_t-2^2',
       'l_NR_ARTICLES_IN_CAT', 'l_NR_ARTICLES_IN_CAT^2',
       'l_NR_ARTICLES_IN_CAT_2', 'l_NR_ARTICLES_IN_CAT_2^2', 'l_WEEK_NR',
       'l_WEEK_NR^2', 'AVG unavailability_perc', 'AVG unavailability_perc^2']]),axis=1, sort=False)

In [ ]:
# adding holiday dummy variable
listofHolidays=["2020-12-28",
                "2021-01-04",
                "2021-03-29",
                "2021-05-03",
                "2021-07-26",
                "2021-08-02",
                "2021-08-09",
                "2021-08-16",
                "2021-12-27",
                "2022-01-03",
                "2022-04-11",
                "2022-05-02",
                "2022-07-25",
                "2022-08-01",
                "2022-08-08",
                "2022-08-15",
                "2022-12-26"
                ]
df_log["HOLIDAY"]=(df_log['WEEK_START_DATE'].isin(listofHolidays))*1

In [ ]:
#dropping outliers
df=df[~(df['WEEK_START_DATE'].isin(["2021-09-06", "2021-12-13"]))] # find date of outliers bij making a boxplot (in f.e. tableau: then you can easily spot the date of the outliers)
